In [1]:
import logging
import sys
import os

import zipfile

import importlib

from six.moves import urllib
from shutil import copy2

import matplotlib.pyplot as plt

import cv2

import numpy as np

from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

In [2]:
def download(url, dest_directory):
    filename = url.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)

    logging.info("Download URL: {}".format(url))
    logging.info("Download DIR: {}".format(dest_directory))

    def _progress(count, block_size, total_size):
                prog = float(count * block_size) / float(total_size) * 100.0
                sys.stdout.write('\r>> Downloading %s %.1f%%' %
                                 (filename, prog))
                sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(url, filepath,
                                             reporthook=_progress)
    print()
    return filepath


importlib.reload(logging)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Download KITTI DATA

kitti_data_url = 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip'

data_dir = './data'

data_road_zip = './data/data_road.zip'


if not os.path.exists(data_road_zip):
    if kitti_data_url == '':
        logging.error("Data URL for Kitti Data not provided.")
        url = "http://www.cvlibs.net/download.php?file=data_road.zip"
        logging.error("Please visit: {}".format(url))
        logging.error("and request Kitti Download link.")
        logging.error("Rerun scipt using"
                      "'python download_data.py' --kitti_url [url]")
        exit(1)
    if not kitti_data_url[-19:] == 'kitti/data_road.zip':
        logging.error("Wrong url.")
        url = "http://www.cvlibs.net/download.php?file=data_road.zip"
        logging.error("Please visit: {}".format(url))
        logging.error("and request Kitti Download link.")
        logging.error("Rerun scipt using"
                      "'python download_data.py' --kitti_url [url]")
        exit(1)
    else:
        logging.info("Downloading Kitti Road Data.")
        download(kitti_data_url, data_dir)

# Extract and prepare KITTI DATA
logging.info("Extracting kitti_road data.")
zipfile.ZipFile(data_road_zip, 'r').extractall(data_dir)
kitti_road_dir = os.path.join(data_dir, 'data_road/')

#logging.info("Preparing kitti_road data.")

train_txt = "data/train3.txt"
val_txt = "data/val3.txt"
testing_txt = "data/testing.txt"

sys.stdout.flush()

#copy2(train_txt, kitti_road_dir)
#copy2(val_txt, kitti_road_dir)
#copy2(testing_txt, kitti_road_dir)

#logging.info("All data have been downloaded successful.")

INFO:root:Extracting kitti_road data.


In [6]:
with open(train_txt, 'r') as f:
    for line in f:
        names = line.split()
        #logging.info(names[0])
        #logging.info(names[1])        
        # load the image and apply SLIC and extract (approximately)
        
        # the supplied number of segments
        image = cv2.imread(os.path.join(kitti_road_dir, names[0]))
        segments = slic(img_as_float(image), n_segments = 200, sigma = 5)

        image2 = cv2.imread(os.path.join(kitti_road_dir, names[1]))
        magenta = np.array([255, 0, 255], dtype=np.uint8)
        #mask2 = np.zeros(image.shape[:2], dtype = "uint8")
        #mask2[image2 == magenta] = 255
        
        #data = np.array(image2)
        #rgb = data[:,:,:3]
        #mask2 = np.all(rgb == magenta, axis = -1)
        
        #lower_magenta = np.array([254,0,254])
        #upper_magenta = np.array([255,1,255])
        #mask2 = cv2.inRange(image2 , lower_magenta, upper_magenta)        
        
        # ok
        mask2 = np.all(image2 == magenta, axis = -1)
        #plt.imshow(mask2)
        
        # show the output of SLIC
        #fig = plt.figure("Superpixels")
        #ax = fig.add_subplot(1, 1, 1)
        #ax.imshow(mark_boundaries(img_as_float(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), segments))
        #plt.axis("off")
        #plt.show()
        # loop over the unique segment values
        for (i, segVal) in enumerate(np.unique(segments)):
            # construct a mask for the segment
            #print("[x] inspecting segment %d" % (i))
            
            mask = np.zeros(image.shape[:2], dtype = "uint8")
            mask[segments == segVal] = 255
            
            m = cv2.moments(mask, True);
            print("centroid coordinates %d : %d" % (m["m10"]/m["m00"], m["m01"]/m["m00"]))
            
            intersection = np.logical_and(mask, mask2)
            
            if intersection.any():
            
            # show the masked region
            #v2.imshow("Mask", mask)
            #v2.imshow("Applied", cv2.bitwise_and(image, image, mask = mask))
            
                #m = cv2.moments(mask, True);
                #print("centroid coordinates %d : %d" % (m["m10"]/m["m00"], m["m01"]/m["m00"]))
            
                img = cv2.bitwise_and(image, image, mask = mask)
                #plt.imshow(img)
                #break
        break


centroid coordinates 44 : 32
centroid coordinates 105 : 25
centroid coordinates 161 : 28
centroid coordinates 214 : 32
centroid coordinates 262 : 27
centroid coordinates 313 : 24
centroid coordinates 362 : 31
centroid coordinates 406 : 16
centroid coordinates 467 : 20
centroid coordinates 551 : 34
centroid coordinates 568 : 16
centroid coordinates 636 : 17
centroid coordinates 698 : 26
centroid coordinates 756 : 25
centroid coordinates 799 : 20
centroid coordinates 850 : 28
centroid coordinates 900 : 27
centroid coordinates 945 : 23
centroid coordinates 1009 : 39
centroid coordinates 1045 : 29
centroid coordinates 1070 : 45
centroid coordinates 1107 : 32
centroid coordinates 1180 : 17
centroid coordinates 1217 : 37
centroid coordinates 729 : 86
centroid coordinates 1157 : 52
centroid coordinates 637 : 61
centroid coordinates 90 : 83
centroid coordinates 591 : 82
centroid coordinates 407 : 73
centroid coordinates 456 : 63
centroid coordinates 769 : 71
centroid coordinates 684 : 78
centr

In [7]:
import tensorflow as tf; print(tf.__version__)

C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\pyth

1.14.0


C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-pac